In [1]:
# from keras.utils import get_file
# dataset = get_file(
#     fname = "aclImdb.tar.gz",
#     origin = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
#     extract = True
# )

# print("下載位置：", dataset)

In [2]:
import glob
import textwrap

fn = glob.glob("C:/Users/Chien Hua Lo/.keras/datasets/aclImdb/train/pos/*")[0]

with open(fn, "r", encoding = "utf-8") as f:
    review = f.read()
    
print("\n".join(textwrap.wrap(review, width=100)))

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life,
such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's
satire is much closer to reality than is "Teachers". The scramble to survive financially, the
insightful students who can see right through their pathetic teachers' pomp, the pettiness of the
whole situation, all remind me of the schools I knew and their students. When I saw the episode in
which a student repeatedly tried to burn down the school, I immediately recalled ......... at
.......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome
to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What
a pity that it isn't!


In [3]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Flatten, Dense, Dropout

model = Sequential()
# 1 padding + 2000個詞 = 2001
model.add(Embedding(2001, 64, input_length=200))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Using TensorFlow backend.
W0113 11:37:49.995262  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0113 11:37:49.997257  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0113 11:37:50.000250  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0113 11:37:50.036157  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0113 11:37:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 64)           128064    
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               3277056   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 3,405,377
Trainable params: 3,405,377
Non-trainable params: 0
_________________________________________________________________


In [4]:
import os
import pandas as pd

def get_content(dirname, target):
    fn = glob.glob("{}/*".format(dirname))
    content = []
    targets = [target] * len(fn)
    for fp in fn:
        with open(fp, "r", encoding="utf-8") as f:
            content.append(f.read())
    return (content, targets)

dirname = "C:/Users/Chien Hua Lo/.keras/datasets"
neg_path = os.path.join(dirname, "aclImdb", "train", "neg")
pos_path = os.path.join(dirname, "aclImdb", "train", "pos")

neg_content, neg_target = get_content(neg_path, 0)
pos_content, pos_target = get_content(pos_path, 1)

train_df = pd.DataFrame({
    "content":neg_content + pos_content,
    "target":neg_target + pos_target
})
train_df

,content,target
0,Story of a man who has unnatural feelings for ...,0
1,Airport '77 starts as a brand new luxury 747 p...,0
2,This film lacked something I couldn't put my f...,0
3,"Sorry everyone,,, I know this is supposed to b...",0
4,When I was little my parents took me along to ...,0
...,...,...
24995,"Seeing as the vote average was pretty low, and...",1
24996,"The plot had some wretched, unbelievable twist...",1
24997,I am amazed at how this movie(and most others ...,1
24998,A Christmas Together actually came before my t...,1


In [5]:
neg_path = os.path.join(dirname, "aclImdb", "test", "neg")
pos_path = os.path.join(dirname, "aclImdb", "test", "pos")
neg_content, neg_target = get_content(neg_path, 0)
pos_content, pos_target = get_content(pos_path, 1)
test_df = pd.DataFrame({
    "content":neg_content + pos_content,
    "target":neg_target + pos_target
})
test_df

,content,target
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0
...,...,...
24995,I was extraordinarily impressed by this film. ...,1
24996,"Although I'm not a golf fan, I attended a snea...",1
24997,"From the start of ""The Edge Of Love"", the view...",1
24998,"This movie, with all its complexity and subtle...",1


In [6]:
# 文字預處理1:把所有文字化成數字
from keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=2000)
# fit_on_texts只是幫你找出有多少種的字(fit-transform的fit)
tok.fit_on_texts(train_df["content"])
# key:詞 value:等一下要轉換成的數字
# 0這個數字保留, 0是拿來做padding
tok.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'he': 26,
 'be': 27,
 'one': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'who': 34,
 'so': 35,
 'from': 36,
 'like': 37,
 'her': 38,
 'or': 39,
 'just': 40,
 'about': 41,
 "it's": 42,
 'out': 43,
 'has': 44,
 'if': 45,
 'some': 46,
 'there': 47,
 'what': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'she': 56,
 'even': 57,
 'my': 58,
 'would': 59,
 'which': 60,
 'only': 61,
 'story': 62,
 'really': 63,
 'see': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'were': 68,
 'me': 69,
 'well': 70,
 'than': 71,
 'we': 72,
 'much': 73,
 'been': 74,
 'bad': 75,
 'get': 76,
 'will': 77,
 'do': 78,
 'also': 79,
 'into': 80,
 'people': 81,
 'other': 82,
 '

In [7]:
# texts_to_sequences:真的依照剛才做出的表幫你轉換完成所有詞(transform)
x_train = tok.texts_to_sequences(train_df["content"])
x_test = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710
0,62,4,3,129,34,44,1414,15,3.0,514.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,514,14,3,159,1702,6,53,16,138.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,19,139,10,423,273,58,20,30.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,803,313,10,121,11,6,421,5,27.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51,10,13,114,58,843,559,69,364.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,316,14,1,856,13,181,361,2,1.0,189.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,1,111,66,46,1294,1296,187,1,1171.0,197.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,10,241,30,86,11,17,2,88,406.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,3,981,292,162,382,156,58,55,18.0,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 文字預處理2. 截長補短
from keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train, maxlen=200)
x_test_pad = pad_sequences(x_test, maxlen=200)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,0,0,0,0,0,0,0,0,0,...,624,31,702,84,702,378,2,67,27,107
1,866,140,3,747,286,531,602,4,14,870,...,1702,40,1609,27,11,354,39,1474,31,1
2,0,0,0,0,0,0,0,0,0,0,...,852,9,13,15,32,731,35,1906,15,620
3,0,0,0,0,0,0,0,0,0,0,...,33,507,297,297,238,631,4,11,6,70
4,841,1,19,44,54,696,54,225,2,6,...,22,4,338,895,900,40,11,19,112,572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,2,52,772,5,103,19,7,7,52,1176
24996,0,0,0,0,0,0,0,0,0,0,...,12,13,28,4,447,385,12,68,1669,903
24997,70,14,84,14,6,3,699,12,6,3,...,2,172,17,31,18,145,99,710,1110,205
24998,0,0,0,0,0,0,0,0,0,0,...,96,9,44,46,18,79,46,201,14,70


In [9]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
               metrics=["accuracy"])

W0113 11:38:22.215073  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0113 11:38:22.241003  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0113 11:38:22.250974  5252 deprecation.py:323] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\tensorflow_core\python\ops\nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import numpy as np

check_callback = ModelCheckpoint("model.h5", 
                                 save_best_only=True)

stop_callback = EarlyStopping(patience=3,
                              restore_best_weights=True)

model.fit(x_train_pad, 
          np.array(train_df["target"]),
          batch_size=200,
          epochs=100,
          validation_split=0.1,
          verbose=2,
          callbacks=[check_callback, stop_callback]) 

W0113 11:38:22.527237  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0113 11:38:22.605029  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0113 11:38:22.662876  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0113 11:38:22.667946  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0113 11:38:22.667946  5252 module_wrapper.py:139] From C:\Users\Chien Hua 

Train on 22500 samples, validate on 2500 samples
Epoch 1/100


W0113 11:38:23.871875  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0113 11:38:23.873873  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

W0113 11:38:23.958640  5252 module_wrapper.py:139] From C:\Users\Chien Hua Lo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



 - 12s - loss: 0.5118 - acc: 0.7193 - val_loss: 0.4504 - val_acc: 0.8144
Epoch 2/100
 - 10s - loss: 0.2069 - acc: 0.9193 - val_loss: 0.4502 - val_acc: 0.8192
Epoch 3/100
 - 11s - loss: 0.0704 - acc: 0.9810 - val_loss: 0.5262 - val_acc: 0.8304
Epoch 4/100
 - 10s - loss: 0.0161 - acc: 0.9979 - val_loss: 0.7139 - val_acc: 0.8168
Epoch 5/100
 - 11s - loss: 0.0043 - acc: 0.9998 - val_loss: 0.7656 - val_acc: 0.8156


In [11]:
model.evaluate(x_test_pad, test_df["target"])

25000/25000 [==============================] - 4s 159us/step


[0.3459704518818855, 0.85132]

In [12]:
# 部分第一招: set_weights
partial1 = Sequential()

# 1 padding + 2000個詞 = 2001
partial1.add(Embedding(2001, 64))

w = model.layers[0].get_weights()
partial1.layers[0].set_weights(w)
partial1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          128064    
Total params: 128,064
Trainable params: 128,064
Non-trainable params: 0
_________________________________________________________________


In [18]:
# the轉化的結果
c = int(input("你輸入一個數字:[1-2000]:"))
print("你輸入的是:", tok.index_word[c])
en = partial1.predict([[c]])[0][0]
print("轉換的語意:", en)

你輸入一個數字:[1-2000]:1
你輸入的是: the
[[1]]
轉換的語意: [ 0.01501018  0.01501249 -0.00494213 -0.04009591 -0.01469069  0.03745698
  0.01977737  0.02488961 -0.0097616  -0.02428296 -0.02847645  0.01497149
 -0.00751056 -0.00533757  0.00240502  0.03542307  0.00914211 -0.05248231
  0.00398042 -0.00492906 -0.01623934 -0.00025995  0.01628004 -0.02801657
  0.04156414  0.00635069 -0.02500909  0.01108127 -0.02313392  0.0221412
  0.01005682 -0.01709567  0.00151705 -0.03534401 -0.00899433 -0.01354634
 -0.04044966  0.00361379 -0.00946301  0.0009691  -0.03629136 -0.00718914
  0.00424109 -0.03282636  0.01210096 -0.02104425 -0.03945764 -0.01844551
 -0.04169448 -0.02836829 -0.00536284 -0.00496952  0.01402532  0.00151476
 -0.00901362 -0.01296506 -0.02757464 -0.01862547 -0.00398369 -0.01517392
  0.01247989  0.00098003  0.00555959 -0.02458748]


In [14]:
# 部分: 第二招
from keras.models import Model
partial2 = Model(inputs=model.input, 
                 outputs=model.layers[0].output)
partial2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1_input (InputLaye (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 64)           128064    
Total params: 128,064
Trainable params: 128,064
Non-trainable params: 0
_________________________________________________________________


In [46]:
# 這裡比較麻煩, shape = (batch, 200)
test = np.array([[1] + [0] * 199])
partial2.predict(test)[0][0]

array([ 0.01501018,  0.01501249, -0.00494213, -0.04009591, -0.01469069,
        0.03745698,  0.01977737,  0.02488961, -0.0097616 , -0.02428296,
       -0.02847645,  0.01497149, -0.00751056, -0.00533757,  0.00240502,
        0.03542307,  0.00914211, -0.05248231,  0.00398042, -0.00492906,
       -0.01623934, -0.00025995,  0.01628004, -0.02801657,  0.04156414,
        0.00635069, -0.02500909,  0.01108127, -0.02313392,  0.0221412 ,
        0.01005682, -0.01709567,  0.00151705, -0.03534401, -0.00899433,
       -0.01354634, -0.04044966,  0.00361379, -0.00946301,  0.0009691 ,
       -0.03629136, -0.00718914,  0.00424109, -0.03282636,  0.01210096,
       -0.02104425, -0.03945764, -0.01844551, -0.04169448, -0.02836829,
       -0.00536284, -0.00496952,  0.01402532,  0.00151476, -0.00901362,
       -0.01296506, -0.02757464, -0.01862547, -0.00398369, -0.01517392,
        0.01247989,  0.00098003,  0.00555959, -0.02458748], dtype=float32)